In [1]:
from transformers import pipeline

# 1. Load the free, pre-trained model
print("Loading model... (this may take a moment)")
qa_pipeline = pipeline(
    'question-answering',
    model='deepset/roberta-base-squad2',
    tokenizer='deepset/roberta-base-squad2'
)

# 2. Define Context and Question
context_text = """
The Amazon rainforest covers most of the Amazon basin of South America.
This basin encompasses 7,000,000 km2. This region includes territory belonging to nine nations.
"""

question_text = "What is the total area of the Amazon basin?"

# 3. Get Answer
input_data = {
    'question': question_text,
    'context': context_text
}

res = qa_pipeline(input_data)

print("-" * 30)
print(f"Question: {question_text}")
print(f"Answer: {res['answer']}")
print(f"Confidence: {res['score']:.4f}")


Loading model... (this may take a moment)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


------------------------------
Question: What is the total area of the Amazon basin?
Answer: 7,000,000 km2
Confidence: 0.9191


In [2]:
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)

# --- MEMORY CLEANUP ---
# Clear cache before starting to avoid lingering memory issues
torch.cuda.empty_cache()
gc.collect()

# 1. SETUP: Configuration
model_checkpoint = "roberta-base"
max_length = 384
doc_stride = 128

# 2. DATA: Load Dataset
# Note: If this still crashes, reduce the split size or use streaming=True (advanced)
print("Loading dataset...")
dataset = load_dataset("squad", split="train[:500]") # Keeping it small for demo validity
dataset = dataset.train_test_split(test_size=0.1)

# 3. TOKENIZATION
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def prepare_train_features(examples):
    # Pad to max_length ensures uniform tensor size, essential for the model structure
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

print("Processing data...")
# BATCHED MAPPING: reducing batch size here helps SYSTEM RAM
tokenized_datasets = dataset.map(
    prepare_train_features,
    batched=True,
    batch_size=100, # Process smaller chunks to save RAM
    remove_columns=dataset["train"].column_names
)

# --- OPTIMIZATION: Free up System RAM ---
# We no longer need the raw text dataset, so we delete it.
del dataset
gc.collect()

# 4. MODEL
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# 5. TRAINING ARGUMENTS (OPTIMIZED FOR MEMORY)
args = TrainingArguments(
    output_dir="roberta-finetuned-squad-optimized",
    eval_strategy="epoch",
    save_strategy="epoch",

    # --- MEMORY OPTIMIZATIONS START ---
    per_device_train_batch_size=4,   # Reduced from 16 to 4 (Fixes OOM)
    per_device_eval_batch_size=4,    # Reduced eval batch size
    gradient_accumulation_steps=4,   # 4 batch size * 4 steps = 16 effective batch size
    fp16=True,                       # Use Mixed Precision (Huge VRAM saver)
    gradient_checkpointing=True,     # Recompute activations (Slower but saves VRAM)
    # ----------------------------------

    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.1,
    logging_steps=5,
    use_cpu=False
)

data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# 6. RUN
print("Starting optimized training...")
trainer.train()

# 7. SAVE
save_path = "./my_optimized_qa_model"
print(f"Saving model to {save_path}...")
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print("Done!")

Loading dataset...


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: varshinimurugan99409 (varshinimurugan99409-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,4.076900,4.033034
2,3.365400,3.146362


Saving model to ./my_optimized_qa_model...
Done!


In [8]:
import torch
from transformers import pipeline

model_path = "./my_optimized_qa_model"

try:
    qa_pipeline = pipeline(
        "question-answering",
        model=model_path,
        tokenizer=model_path,
        device=device
    )
    print("Model loaded successfully!\n")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Did you complete the training step? Ensure the folder './my_optimized_qa_model' exists.")
    exit()

test_cases = [
    {
        "category": "Science / Geography",
        "context": """
        The Amazon rainforest, alternatively, the Amazon Jungle, also known in English as Amazonia,
        is a moist broadleaf tropical rainforest in the Amazon biome that covers most of the Amazon basin of South America.
        This basin encompasses 7,000,000 km2 (2,700,000 sq mi), of which 5,500,000 km2 (2,100,000 sq mi)
        are covered by the rainforest. This region includes territory belonging to nine nations.
        """,
        "question": "How many nations does the Amazon region include?"
    }
]

for i, test in enumerate(test_cases):
    print(f"\nCategory: {test['category']}")

    result = qa_pipeline(question=test['question'], context=test['context'])

    print(f"Question:  {test['question']}")
    print(f"Answer:    {result['answer']}")
    print(f"Confidence:{result['score']:.4f}")


Device set to use cuda:0


Model loaded successfully!


Category: Science / Geography
Question:  How many nations does the Amazon region include?
Answer:    nine
Confidence:0.0082
